# ITCS 6162: Data Mining - Programming Assignment

**In this assignment, you will explore data analysis, recommendation algorithms, and graph-based techniques using the MovieLens dataset. Your tasks will range from basic data exploration to advanced recommendation models, including:**
- Data manipulation with pandas
- User-item collaborative filtering
- Similarity-based recommendation models
- A Pixie-inspired Graph-based recommendation using adjacency lists with weighted random walks (without using NetworkX)


#### **Dataset Files:**
- **`u.data`**: User-movie ratings (`user_id  movie_id  rating  timestamp`)
- **`u.item`**: Movie metadata (`movie_id | title | release date | IMDB_website`)
- **`u.user`**: User demographics (`user_id | age | gender | occupation | zip_code`)

## **Part 1: Exploring and Cleaning Data**

### Inspecting the Dataset Format

The dataset is not in a traditional CSV format. To examine its structure, use the following shell command to display the first 10 lines of the file:

```sh
!head <file_name>


**In the cells given below. Write the code to read the files.**

In [1]:
!head data/u.data


196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013


In [2]:
!head data/u.item

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

In [3]:
!head data/u.user


1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213
6|42|M|executive|98101
7|57|M|administrator|91344
8|36|M|administrator|05201
9|29|M|student|01002
10|53|M|lawyer|90703


#### Loading the Dataset with Pandas

Use **pandas** to load the dataset into a DataFrame for analysis. Follow these steps:  

1. Import the necessary library: `pandas`.  
2. Use `pd.read_csv()` (or an appropriate function) to read the dataset file.  
3. Ensure the dataset is loaded with the correct delimiter (e.g., `','`, `'\t'`,`'|'` , or another separator if needed).  
4. Select and display the first few rows using `.head()`.

Ensure that:  

- The `ratings` dataset is read from `"u.data"` using tab (`'\t'`) as a separator and column names (`"user_id"`, `"movie_id"`, `"rating"` and `"timestamp"`).  
- The `movies` dataset is read from `"u.item"` using `'|'` as a separator, use columns (`0`, `1`, `2`), encoding (`"latin-1"`) and name the columns (`movie_id`, `title`, and `release_date`).  
- The `users` dataset is read from `"u.user"` using `'|'` as a separator, use columns (`0`, `1`, `2`, `3`) and name the columns (`user_id`, `age`, `gender`, and `occupation`).

In [4]:
import pandas as pd

In [5]:
ratings = pd.read_csv('data/u.data', sep = '\t', header = None, names = ['user_id', 'movie_id', 'rating', 'timestamp'])
ratings


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [6]:
movies = pd.read_csv('data/u.item', sep = '|', usecols = [0, 1, 2], encoding = 'latin-1', names = ['movie_id', 'title', 'release_date'])
movies

,movie_id,title,release_date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995
...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998
1678,1679,B. Monkey (1998),06-Feb-1998
1679,1680,Sliding Doors (1998),01-Jan-1998
1680,1681,You So Crazy (1994),01-Jan-1994


In [7]:
users = pd.read_csv('data/u.user', sep = '|', usecols = [0, 1, 2, 3], names = ['user_id', 'age', 'gender', 'occupation'])
users


,user_id,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other
...,...,...,...,...
938,939,26,F,student
939,940,32,M,administrator
940,941,20,M,student
941,942,48,F,librarian


**Note:** As a **Bonus** task save the `ratings`, `movies` and `users` dataframe created into a `.csv` file format. <br>
**Hint:** Use the `to_csv()` function in pandas to save these DataFrames as CSV files.

In [8]:
ratings.to_csv('data/ratings.csv')


In [9]:
movies.to_csv('data/movies.csv')

In [10]:
users.to_csv('data/users.csv')

**Display the first 10 rows of each file.**

In [11]:
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [12]:
movies.head(10)

,movie_id,title,release_date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995
6,7,Twelve Monkeys (1995),01-Jan-1995
7,8,Babe (1995),01-Jan-1995
8,9,Dead Man Walking (1995),01-Jan-1995
9,10,Richard III (1995),22-Jan-1996


In [13]:
users.head(10)

,user_id,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other
5,6,42,M,executive
6,7,57,M,administrator
7,8,36,M,administrator
8,9,29,M,student
9,10,53,M,lawyer


### Data Cleaning and Exploration with Pandas  

After loading the dataset, it’s important to clean and explore the data to ensure consistency and accuracy. Below are key **pandas** functions for cleaning and understanding the dataset.

#### 1. Handle Missing Values  
- `df.dropna()` – Removes rows with missing values.  
- `df.fillna(value)` – Fills missing values with a specified value.  

#### 2. Remove Duplicates  
- `df.drop_duplicates()` – Drops duplicate rows from the dataset.  

#### 3. Handle Incorrect Data Types  
- `df.astype(dtype)` – Converts columns to the appropriate data type.  

#### 4. Filter Outliers (if applicable)  
- `df[df['column_name'] > threshold]` – Filters rows based on a condition.  

#### 5. Rename Columns (if needed)  
- `df.rename(columns={'old_name': 'new_name'})` – Renames columns for clarity.  

#### 6. Reset Index  
- `df.reset_index(drop=True, inplace=True)` – Resets the index after cleaning.  

### Data Exploration Functions  

To better understand the dataset, use these **pandas** functions:  

- `df.shape` – Returns the number of rows and columns in the dataset.  
- `df.nunique()` – Displays the number of unique values in each column.  
- `df['column_name'].unique()` – Returns unique values in a specific column.  

**Example Usage in Pandas:**  
```python
import pandas as pd

# Load dataset
df = pd.read_csv("your_file.csv")

# Drop missing values
df_cleaned = df.dropna()

# Remove duplicate rows
df_cleaned = df_cleaned.drop_duplicates()

# Convert 'timestamp' column to datetime format
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'])

# Display dataset shape
print("Dataset shape:", df_cleaned.shape)

# Display number of unique values in each column
print("Unique values per column:\n", df_cleaned.nunique())

# Display unique movie IDs
print("Unique movie IDs:", df_cleaned['movie_id'].unique()[:10])  # Show first 10 unique movie IDs


**Note:** The functions mentioned above are some of the widely used **pandas** functions for data cleaning and exploration. However, it is not necessary that all of these functions will be required in the exercises below. Use them as needed based on the dataset and the specific tasks.

**Convert Timestamps into readable dates.**

In [14]:
movies['release_date'] = pd.to_datetime(movies['release_date'])
movies

,movie_id,title,release_date
0,1,Toy Story (1995),1995-01-01
1,2,GoldenEye (1995),1995-01-01
2,3,Four Rooms (1995),1995-01-01
3,4,Get Shorty (1995),1995-01-01
4,5,Copycat (1995),1995-01-01
...,...,...,...
1677,1678,Mat' i syn (1997),1998-02-06
1678,1679,B. Monkey (1998),1998-02-06
1679,1680,Sliding Doors (1998),1998-01-01
1680,1681,You So Crazy (1994),1994-01-01


**Check for missing values**

In [15]:
ratings.isna().any()

user_id      False
movie_id     False
rating       False
timestamp    False
dtype: bool

In [16]:
movies.isna().any()

movie_id        False
title           False
release_date     True
dtype: bool

In [17]:
movies.dropna(inplace = True)

In [18]:
users.isna().any()

user_id       False
age           False
gender        False
occupation    False
dtype: bool

**Remove/check for duplicates**

In [19]:
ratings.duplicated().any()

False

In [20]:
movies.duplicated().any()

False

In [21]:
users.duplicated().any()

False

**Print the total number of users, movies, and ratings.**

In [22]:
print(f"Total Users: {users.shape[0]}")
print(f"Total Movies: {movies.shape[0]}")
print(f"Total Ratings: {ratings.shape[0]}")

Total Users: 943
Total Movies: 1681
Total Ratings: 100000


## **Part 2: Collaborative Filtering-Based Recommendation**

### **Create a User-Item Matrix**

#### Instructions for Creating a User-Movie Rating Matrix

In this exercise, you will create a user-movie rating matrix using **pandas**. This matrix will represent the ratings that users have given to different movies.

1. **Dataset Overview**:  
   The dataset has already been loaded. It includes the following key columns:
   - `user_id`: The ID of the user.
   - `movie_id`: The ID of the movie.
   - `ratings`: The rating the user gave to the movie.

2. **Create the User-Movie Rating Matrix**:  
   Use the **`pivot()`** function in **pandas** to reshape the data. Your goal is to create a matrix where:
   - Each **row** represents a **user**.
   - Each **column** represents a **movie**.
   - Each **cell** contains the **rating** that the user has given to the movie.

   Specify the following parameters for the `pivot()` function:
   - **`index`**: The `user_id` column (this will define the rows).
   - **`columns`**: The `movie_id` column (this will define the columns).
   - **`values`**: The `rating` column (this will fill the matrix with ratings).

3. **Inspect the Matrix**:  
   After creating the matrix, examine the first few rows of the resulting matrix to ensure it has been constructed correctly.

4. **Handle Missing Values**:  
   It's likely that some users have not rated every movie, resulting in `NaN` values in the matrix. You will need to handle these missing values. Consider the following options:
   - **Fill with 0**: If you wish to represent missing ratings as zeros (indicating no rating).
   - **Fill with the average rating**: Alternatively, replace missing values with the average rating for each movie.

**Create the user-movie rating matrix using the `pivot()` function.**

In [23]:
user_movie_matrix = ratings.pivot(index = 'user_id', columns = 'movie_id', values = 'rating')

**Display the matrix to verify the transformation.**

In [24]:
user_movie_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
user_movie_matrix.fillna(0, inplace = True)
# user_movie_matrix.fillna(user_movie_matrix.mean(), inplace = True)
user_movie_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### **User-Based Collaborative Filtering Recommender System**

#### **Objective**
In this task, you will implement a **user-based collaborative filtering** movie recommendation system using the **Movie dataset**. The goal is to recommend movies to a user based on the preferences of similar users.

##### **Step 1: Import Required Libraries**
Before starting, ensure you have the necessary libraries installed. Use the following imports:

```python
import pandas as pd  # For handling data
import numpy as np   # For numerical computations
from sklearn.metrics.pairwise import cosine_similarity  # For computing user similarity
```

In [26]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

##### **Step 2: Compute User-User Similarity**
- We will use **cosine similarity** to measure how similar each pair of users is based on their movie ratings.
- Since `cosine_similarity` does not handle missing values (NaN), replace them with `0` before computation.

##### **Instructions:**
1. Fill missing values with `0` using `.fillna(0)`.
2. Compute similarity using `cosine_similarity()`.
3. Convert the result into a **Pandas DataFrame**, with users as both row and column labels.

##### **Hint:**  
You can achieve this using the following approach:

```python
user_similarity = cosine_similarity(user_movie_matrix.fillna(0))
user_sim_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)
```

In [27]:
def get_similar_user_matrix() -> pd.DataFrame:
    sim = cosine_similarity(user_movie_matrix)   
    return pd.DataFrame(sim, index = user_movie_matrix.index, columns = user_movie_matrix.index)

##### **Step 3: Implement the Recommendation Function**
Now, implement the function `recommend_movies_for_user(user_id, num=5)` to recommend movies for a given user.

##### **Function Inputs:**
- `user_id`: The target user for whom we need recommendations.
- `num`: The number of movies to recommend (default is 5).

##### **Function Steps:**
1. Find **similar users**:
   - Retrieve the similarity scores for the given `user_id`.
   - Sort them in **descending** order (highest similarity first).
   - Exclude the user themselves.
   
2. Get the **movie ratings** from these similar users.

3. Compute the **average rating** for each movie based on these users' preferences.

4. Sort the movies in **descending order** based on the computed average ratings.

5. Retrieve the **top `num` recommended movies**.

6. Map **movie IDs** to their **titles** using the `movies` DataFrame.

7. Return the results as a **Pandas DataFrame** with rankings.

In [28]:
def recommend_movies_for_user(user_id: int, num: int = 5) -> pd.DataFrame:
    # 1: get similar users
    similar_user_matrix = get_similar_user_matrix()
    
    # 1a: sort in descending order
    similar_user_matrix = similar_user_matrix.loc[user_id].sort_values(ascending = False)

    # 1b: exclude user_id
    similar_user_matrix = similar_user_matrix.drop(user_id) 

    # 2: get similar users ratings
    similar_user_ratings = user_movie_matrix.loc[similar_user_matrix.index]

    # 3: Compute average rating for each movie
    average_ratings = similar_user_ratings.mean()

    # 4: Sort movies in descending order based on computed averages
    average_ratings = average_ratings.sort_values(ascending = False)

    # 5: Retrieve top num recommendations
    top_recs = average_ratings.head(num)

    # 6: Map movie ids to titles using movies dataframe
    movie_names = movies[movies['movie_id'].isin(top_recs.index)]['title']

    # 7: Return results as dataframe 
    return pd.DataFrame({
        'Ranking': range(1, num + 1),
        'Movie Name': movie_names.values
    })
 

##### **Step 4: Return the Final Recommendation List**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

##### **Hint:** Your final DataFrame should be created like this:
```python
result_df = pd.DataFrame({
    'Ranking': range(1, num+1),
    'Movie Name': movie_names     
})
result_df.set_index('Ranking', inplace=True)
```

#### **Example: User-Based Collaborative Filtering**
```python
recommend_movies_for_user(10, num = 5)
```
**Output:**
```
| Ranking | Movie Name                     |
|---------|--------------------------------|
| 1       | In the Company of Men (1997)   |
| 2       | Misérables, Les (1995)         |
| 3       | Thin Blue Line, The (1988)     |
| 4       | Braindead (1992)               |
| 5       | Boys, Les (1997)               |


In [29]:
recommend_movies_for_user(10)

,Ranking,Movie Name
0,1,Star Wars (1977)
1,2,Fargo (1996)
2,3,Raiders of the Lost Ark (1981)
3,4,Return of the Jedi (1983)
4,5,Contact (1997)


### **Item-Based Collaborative Filtering Recommender System**

#### **Objective**
In this task, you will implement an **item-based collaborative filtering** recommendation system using the **Movie dataset**. The goal is to recommend movies similar to a given movie based on user rating patterns.

#### **Step 1: Import Required Libraries**
Although we have done this part already in the previous task but just to emphasize the importance reiterrating this part.

Before starting, ensure you have the necessary libraries installed. Use the following imports:

```python
import pandas as pd  # For handling data
import numpy as np   # For numerical computations
from sklearn.metrics.pairwise import cosine_similarity  # For computing item similarity
```

#### **Step 2: Compute Item-Item Similarity**
- We will use **cosine similarity** to measure how similar each pair of movies is based on their user ratings.
- Since `cosine_similarity` does not handle missing values (NaN), replace them with `0` before computation.
- Unlike user-based filtering, we need to **transpose** (`.T`) the `user_movie_matrix` because we want similarity between movies (columns) instead of users (rows).

##### **Instructions:**
1. Transpose the user-movie matrix using `.T` to make movies the rows.
2. Fill missing values with `0` using `.fillna(0)`.
3. Compute similarity using `cosine_similarity()`.
4. Convert the result into a **Pandas DataFrame**, with movies as both row and column labels.

##### **Hint:**  
You can achieve this using the following approach:

```python
item_similarity = cosine_similarity(user_movie_matrix.T.fillna(0))
item_sim_df = pd.DataFrame(item_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)
```

In [30]:
sim = cosine_similarity(user_movie_matrix.T)
item_sim_df = pd.DataFrame(sim, index = user_movie_matrix.columns, columns = user_movie_matrix.columns)

#### **Step 3: Implement the Recommendation Function**
Now, implement the function `recommend_movies(movie_name, num=5)` to recommend movies similar to a given movie.

##### **Function Inputs:**
- `movie_name`: The target movie for which we need recommendations.
- `num`: The number of similar movies to recommend (default is 5).

##### **Function Steps:**
1. Find the **movie_id** corresponding to the given `movie_name` in the `movies` DataFrame.
2. If the movie is not found, return an appropriate message.
3. Extract the **similarity scores** for this movie from `item_sim_df`.
4. Sort the movies in **descending order** based on similarity (excluding the movie itself).
5. Retrieve the **top `num` similar movies**.
6. Map **movie IDs** to their **titles** using the `movies` DataFrame.
7. Return the results as a **Pandas DataFrame** with rankings.

In [ ]:
def recommend_movies(movie_name: str, num: int = 5) -> pd.DataFrame:
    # 1/2: find movie id from movies df and print message if not there
    try:
        movie_id = movies[movies['title'] == movie_name]['movie_id'].tolist()[0]
    except IndexError:
        raise ValueError(f'Movie "{movie_name}" not found in database') 
    
    # 3: extract similarity scores for this move from sim 
    similarities = item_sim_df.loc[movie_id]

    # 4: sort in descending order
    similarities = similarities.drop(movie_id)
    similarities = similarities.sort_values(ascending = False)
                    
    # 5: retrieve top num similar movies
    top_recs = similarities.head(num)

    # 6: map movie ids to titles
    movie_names = movies[movies['movie_id'].isin(top_recs.index)]['title']

    # 7: return dataframe 
    return pd.DataFrame({
        'Ranking': range(1, num + 1),
        'Movie Name': movie_names
    }).reset_index(drop = True)

#### **Step 4: Return the Final Recommendation List**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

##### **Hint:** Your final DataFrame should be created like this:
```python
result_df = pd.DataFrame({
    'ranking': range(1, num+1),
    'movie_name': movie_names
})
result_df.set_index('ranking', inplace=True)
```

#### **Example: Item-Based Collaborative Filtering**
```python
recommend_movies("Jurassic Park (1993)", num=5)
```
**Output:**
```
| Ranking | Movie Name                               |
|---------|------------------------------------------|
| 1       | Top Gun (1986)                           |
| 2       | Empire Strikes Back, The (1980)          |
| 3       | Raiders of the Lost Ark (1981)           |
| 4       | Indiana Jones and the Last Crusade (1989)|
| 5       | Speed (1994)                             |


In [32]:
recommend_movies("Jurassic Park (1993)")

,Ranking,Movie Name
0,1,Top Gun (1986)
1,2,"Empire Strikes Back, The (1980)"
2,3,Raiders of the Lost Ark (1981)
3,4,Indiana Jones and the Last Crusade (1989)
4,5,Speed (1994)


## **Part 3: Graph-Based Recommender (Pixie-Inspired Algorithm)**

### **Adjacency List**

#### **Objective**
In this task, you will preprocess the Movie dataset and construct a **graph representation** where:
- **Users** are connected to the movies they have rated.
- **Movies** are connected to users who have rated them.
  
This graph structure will help in exploring **user-movie relationships** for recommendations.

#### **Step 1: Merge Ratings with Movie Titles**
Since we have **movie IDs** in the ratings dataset but need human-readable movie titles, we will:
1. Merge the `ratings` DataFrame with the `movies` DataFrame using the `'movie_id'` column.
2. This allows each rating to be associated with a **movie title**.

#### **Hint:**
Use the following Pandas operation to merge:
```python
ratings = ratings.merge(movies, on='movie_id')
```

In [33]:
ratings = ratings.merge(movies, on = 'movie_id')


#### **Step 2: Aggregate Ratings**
Since multiple users may rate the same movie multiple times, we:
1. Group the dataset by `['user_id', 'movie_id', 'title']`.
2. Compute the **mean rating** for each movie by each user.
3. Reset the index to ensure we maintain a clean DataFrame structure.

#### **Hint:**  
Use `groupby()` and `mean()` as follows:
```python
ratings = ratings.groupby(['user_id', 'movie_id', 'title'])['rating'].mean().reset_index()
```

In [34]:
ratings = ratings.groupby(['user_id', 'movie_id', 'title'])['rating'].mean().reset_index()

#### **Step 3: Normalize Ratings**
Since different users have different rating biases, we normalize ratings by:
1. **Computing each user's mean rating**.
2. **Subtracting the mean rating** from each individual rating.

#### **Instructions:**
- Use `groupby('user_id')` to group ratings by users.
- Apply `transform(lambda x: x - x.mean())` to adjust ratings.

#### **Hint:**  
Normalize ratings using:
```python
ratings['rating'] = ratings.groupby('user_id')['rating'].transform(lambda x: x - x.mean())
```
This ensures each user’s ratings are centered around zero, making similarity calculations fairer.

In [35]:
ratings['rating'] = ratings.groupby('user_id')['rating'].transform(lambda x: x - x.mean())

In [36]:
ratings

,user_id,movie_id,title,rating
0,1,1,Toy Story (1995),1.391144
1,1,2,GoldenEye (1995),-0.608856
2,1,3,Four Rooms (1995),0.391144
3,1,4,Get Shorty (1995),-0.608856
4,1,5,Copycat (1995),-0.608856
...,...,...,...,...
99986,943,1067,Bottle Rocket (1996),-1.410714
99987,943,1074,Reality Bites (1994),0.589286
99988,943,1188,Young Guns II (1990),-0.410714
99989,943,1228,Under Siege 2: Dark Territory (1995),-0.410714


#### **Step 4: Construct the Graph Representation**
We represent the user-movie interactions as an **undirected graph** using an **adjacency list**:
- Each **user** is a node connected to movies they rated.
- Each **movie** is a node connected to users who rated it.

#### **Graph Construction Steps:**
1. Initialize an empty dictionary `graph = {}`.
2. Iterate through the **ratings dataset**.
3. For each `user_id` and `movie_id` pair:
   - Add the movie to the user’s set of connections.
   - Add the user to the movie’s set of connections.

#### **Hint:**  
The following code builds the graph:

```python
graph = {}
for _, row in ratings.iterrows():
    user, movie = row['user_id'], row['movie_id']
    if user not in graph:
        graph[user] = set()
    if movie not in graph:
        graph[movie] = set()
    graph[user].add(movie)
    graph[movie].add(user)
```

This results in a **bipartite graph**, where:
- **Users** are connected to multiple movies.
- **Movies** are connected to multiple users.

In [37]:
graph = {}
for _, row in ratings.iterrows():
    user, movie, rating = row['user_id'], row['movie_id'], row['rating']

    if user not in graph:
        graph[user] = set()
    
    if movie not in graph:
        graph[movie] = set()

    graph[user].add((movie, rating))
    graph[movie].add((user, rating))

#### **Step 5: Understanding the Graph**
- **Nodes** in the graph represent **users and movies**.
- **Edges** exist between a user and a movie **if the user has rated the movie**.
- This structure allows us to find **users with similar movie tastes** and **movies frequently watched together**.

#### **Exploring the Graph**
- **Find a user’s rated movies:**  
  ```python
  user_id = 1
  print(graph[user_id])  # Movies rated by user 1
  ```

- **Find users who rated a movie:**  
  ```python
  movie_id = 50
  print(graph[movie_id])  # Users who rated movie 50
  ```

In [ ]:
def explore_graph(user_id: int = None, movie_id: int = None):
    if user_id:
        print(f'User "{user_id}" rated movies: {graph[user_id]}')
    if movie_id:
        print(f'Users who rated "{movie_id}": {graph[movie_id]}')

In [ ]:
explore_graph(1)

User "1" rated movies: {(839, 0.807017543859649), (120, -2.6088560885608856), (292, -0.22580645161290303), (298, 0.9685039370078741), (320, -0.720779220779221), (89, 1.3911439114391144), (715, 1.6167664670658684), (223, 1.3911439114391144), (134, 0.3911439114391144), (505, -0.3660714285714284), (93, 1.65), (17, -0.6088560885608856), (28, 0.3911439114391144), (248, -0.5714285714285716), (162, 0.3911439114391144), (892, 1.0), (934, -1.7011494252873565), (230, 0.3911439114391144), (241, 0.3911439114391144), (135, 0.3911439114391144), (606, 1.0338983050847457), (726, 1.0), (592, 0.1861111111111109), (108, 1.3911439114391144), (253, 1.3911439114391144), (137, -1.3191489361702127), (192, 0.3911439114391144), (203, 0.3911439114391144), (705, 1.289473684210526), (398, 1.2906976744186047), (97, -0.6088560885608856), (142, -1.6088560885608856), (36, -1.6088560885608856), (59, 1.3911439114391144), (535, -0.9357798165137616), (280, 0.4224806201550386), (131, -2.6088560885608856), (195, 1.391143911

In [ ]:
explore_graph(movie_id = 50)

Users who rated "50": {(102, 1.384259259259259), (292, -0.22580645161290303), (298, 0.9685039370078741), (320, 0.27922077922077904), (715, 1.6167664670658684), (505, -0.3660714285714284), (562, 1.4583333333333335), (144, 1.325242718446602), (892, 1.0), (176, 1.3015873015873014), (760, -0.24390243902439046), (878, 0.7313432835820897), (516, 0.9047619047619051), (77, 0.38888888888888884), (270, 0.666666666666667), (185, -0.04166666666666696), (23, 0.36423841059602635), (606, 1.0338983050847457), (774, 1.9419642857142856), (480, 0.2666666666666666), (833, -1.0639097744360901), (890, 1.1428571428571428), (235, 1.021276595744681), (398, 1.2906976744186047), (194, 0.0360655737704918), (597, 1.3658536585365852), (913, 0.47727272727272707), (56, 1.358288770053476), (280, -0.5775193798449614), (782, 0.2112068965517242), (663, 1.4683544303797467), (649, 0.20833333333333348), (72, -1.8175182481751824), (311, 1.204081632653061), (148, 1.0), (771, 0.2857142857142856), (561, -0.025210084033613356), 

### **Implement Weighted Random Walks**

#### **Random Walk-Based Movie Recommendation System (Weighted Pixie)**

#### **Objective**
In this task, you will implement a **random-walk-based recommendation algorithm** using the **Weighted Pixie** method. This technique uses a **user-movie bipartite graph** to recommend movies by simulating a random walk from a given user or movie.

#### **Step 1: Import Required Libraries**
Make sure you have the necessary libraries:

```python
import random  # For random walks
import pandas as pd  # For handling data
```

In [41]:
import random

    #### **Step 2: Implement the Random Walk Algorithm**
    Your task is to **simulate a random walk** from a given starting point in the **bipartite user-movie graph**.

    ##### **Hints for Implementation**
    - Start from **either a user or a movie**.
    - At each step, **randomly move** to a connected node.
    - Keep track of **how many times each movie is visited**.
    - After completing the walk, **rank movies by visit count**.

In [ ]:
def weighted_random_walk(n_steps: int, user_id: int = None, movie_id: int = None) -> dict:
    current_node = movie_id if movie_id is not None else user_id
    visits = {}
    steps_completed = 0

    if not movie_id and not user_id:
        raise ValueError("either user_id or movie_id must be provided")
    
    while steps_completed < n_steps:
        if current_node in movies.index:
            visits[current_node] = visits.get(current_node, 0) + 1
            steps_completed += 1
    
        neighbors = list(graph[current_node])
        nodes = [neighbor[0] for neighbor in neighbors]
        weights = [neighbor[1] for neighbor in neighbors]

        # if sum weights < 1, choose uniformly
        if sum(weights) <= 0:
            current_node = random.choices(nodes)[0]
        else:
            current_node = random.choices(nodes, weights)[0]

    return dict(sorted(list(visits.items()), key = lambda x: x[1], reverse = True))

In [97]:
weighted_random_walk(n_steps = 100, user_id = 5)

{100: 3,
 5: 2,
 291: 2,
 144: 2,
 742: 2,
 409: 2,
 154: 1,
 94: 1,
 176: 1,
 643: 1,
 4: 1,
 352: 1,
 436: 1,
 787: 1,
 350: 1,
 519: 1,
 745: 1,
 663: 1,
 978: 1,
 251: 1,
 158: 1,
 283: 1,
 195: 1,
 933: 1,
 163: 1,
 934: 1,
 887: 1,
 13: 1,
 88: 1,
 712: 1,
 419: 1,
 330: 1,
 58: 1,
 535: 1,
 174: 1,
 554: 1,
 56: 1,
 445: 1,
 184: 1,
 57: 1,
 199: 1,
 426: 1,
 823: 1,
 401: 1,
 805: 1,
 428: 1,
 327: 1,
 302: 1,
 269: 1,
 525: 1,
 747: 1,
 664: 1,
 603: 1,
 99: 1,
 648: 1,
 546: 1,
 255: 1,
 870: 1,
 235: 1,
 457: 1,
 12: 1,
 690: 1,
 79: 1,
 347: 1,
 686: 1,
 127: 1,
 159: 1,
 790: 1,
 880: 1,
 731: 1,
 143: 1,
 339: 1,
 1258: 1,
 393: 1,
 1285: 1,
 642: 1,
 732: 1,
 405: 1,
 1065: 1,
 59: 1,
 845: 1,
 310: 1,
 116: 1,
 148: 1,
 214: 1,
 659: 1,
 253: 1,
 132: 1,
 683: 1,
 321: 1,
 20: 1,
 296: 1,
 1142: 1}

In [99]:
weighted_random_walk(n_steps = 10, movie_id = 100)

{100: 1,
 692: 1,
 487: 1,
 128: 1,
 433: 1,
 476: 1,
 318: 1,
 493: 1,
 274: 1,
 148: 1}

#### **Step 3: Implement User-Based Recommendation**
**Hints:**
- Check if the `user_id` exists in the `graph`.
- Start a loop that runs for `walk_length` steps.
- Randomly pick a **connected node** (user or movie).
- Track how many times each **movie** is visited.
- Sort movies by visit frequency and return the **top N**.

#### **Step 4: Implement Movie-Based Recommendation**
**Hints:**
- Find the `movie_id` corresponding to the given `movie_name`.
- Ensure the movie exists in the `graph`.
- Start a random walk from that movie.
- Follow the same **tracking and ranking** process as the user-based version.

**Note:**  
**Your task:** Implement a function `weighted_pixie_recommend(user_id, walk_length=15, num=5)` or `weighted_pixie_recommend(movie_name, walk_length=15, num=5)`.  
**Implement either Step 3 or Step 4.**

In [78]:
def weighted_pixie_recommend(movie_name: str, walk_length: int = 15, num: int = 5) -> pd.DataFrame:
    # 1: find movie_id given movie_name
    try:
        movie_id = movies[movies['title'] == movie_name]['movie_id'].tolist()[0]
    except IndexError:
        raise ValueError(f'Movie "{movie_name}" not found in database') 

    # 2: make sure it exists in the graph
    if movie_id not in graph:
        raise ValueError(f'Movie "{movie_name}" not found in graph.')

    # 3/4/5: random walk
    ranked = weighted_random_walk(walk_length, movie_id = movie_id)
    ranked.pop(movie_id, None) # remove existing movie_id from ranked if inside 
    top_n_movie_ids = list(ranked.keys())[:num]
    movie_names = [movies.loc[movies['movie_id'] == m_id, 'title'].iloc[0] for m_id in top_n_movie_ids]

    # 6: return dataframe
    return pd.DataFrame({
        'Ranking': range(1, num + 1),
        'Movie Name': movie_names
    })

#### **Step 5: Running Your Recommendation System**
Once your function is implemented, test it by calling:

##### **Example: User-Based Recommendation**
```python
weighted_pixie_recommend(1, walk_length=15, num=5)
```
| Ranking | Movie Name                     |
|---------|--------------------------------|
| 1       | My Own Private Idaho (1991)   |
| 2       | Aladdin (1992)                |
| 3       | 12 Angry Men (1957)           |
| 4       | Happy Gilmore (1996)          |
| 5       | Copycat (1995)                |


##### **Example: Movie-Based Recommendation**
```python
weighted_pixie_recommend("Jurassic Park (1993)", walk_length=10, num=5)
```
| Ranking | Movie Name                           |
|---------|-------------------------------------|
| 1       | Rear Window (1954)                 |
| 2       | Great Dictator, The (1940)         |
| 3       | Field of Dreams (1989)             |
| 4       | Casablanca (1942)                  |
| 5       | Nightmare Before Christmas, The (1993) |


#### **Step 6: Understanding the Results**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

Each movie is ranked based on **how frequently it was visited** during the walk.

#### **Experiment with Different Parameters**
- Try different **`walk_length`** values and observe how it changes recommendations.
- Adjust the number of recommended movies (`num`).

In [84]:
weighted_pixie_recommend("Jurassic Park (1993)", walk_length = 10)

,Ranking,Movie Name
0,1,Jackie Chan's First Strike (1996)
1,2,"Devil's Advocate, The (1997)"
2,3,"Secret of Roan Inish, The (1994)"
3,4,Eve's Bayou (1997)
4,5,Sabrina (1954)


In [91]:
weighted_pixie_recommend("Jurassic Park (1993)", walk_length = 100, num = 10)

,Ranking,Movie Name
0,1,Chasing Amy (1997)
1,2,True Romance (1993)
2,3,Money Talks (1997)
3,4,"Sound of Music, The (1965)"
4,5,Titanic (1997)
5,6,Amityville 1992: It's About Time (1992)
6,7,Independence Day (ID4) (1996)
7,8,Citizen Kane (1941)
8,9,"Wrong Trousers, The (1993)"
9,10,Aliens (1986)


In [92]:
weighted_pixie_recommend("Jurassic Park (1993)", walk_length = 1000, num = 10)

,Ranking,Movie Name
0,1,Mighty Aphrodite (1995)
1,2,Fargo (1996)
2,3,Mission: Impossible (1996)
3,4,"Princess Bride, The (1987)"
4,5,Raiders of the Lost Ark (1981)
5,6,Mary Shelley's Frankenstein (1994)
6,7,Old Yeller (1957)
7,8,Titanic (1997)
8,9,Ulee's Gold (1997)
9,10,Sling Blade (1996)


In [93]:
weighted_pixie_recommend("Jurassic Park (1993)", walk_length = 10000, num = 20)

,Ranking,Movie Name
0,1,Mission: Impossible (1996)
1,2,"Full Monty, The (1997)"
2,3,"Godfather, The (1972)"
3,4,Fargo (1996)
4,5,Return of the Jedi (1983)
5,6,Leaving Las Vegas (1995)
6,7,Mighty Aphrodite (1995)
7,8,Star Trek: First Contact (1996)
8,9,Raiders of the Lost Ark (1981)
9,10,Titanic (1997)


---

## **Submission Requirements:**

To successfully complete this assignment, ensure that you submit the following:


### **1. Jupyter Notebook Submission**
- Submit a **fully completed Jupyter Notebook** that includes:
  - **All implemented recommendation functions** (user-based, item-based, and random walk-based recommendations).
  - **Code explanations** in markdown cells to describe each step.
  - **Results and insights** from running your recommendation models.


### **2. Explanation of Pixie-Inspired Algorithms (3-5 Paragraphs)**
- Write a **detailed explanation** of **Pixie-inspired random walk algorithms** used for recommendations.
- Your explanation should cover:
  - What **Pixie-inspired recommendation systems** are.
  - How **random walks** help in identifying relevant recommendations.
  - Any real-world applications of such algorithms in industry.


### **3. Report for the Submitted Notebook**
Your report should be structured as follows:

#### **Title: Movie Recommendation System Report**

#### **1. Introduction**
- Briefly introduce **movie recommendation systems** and why they are important.
- Explain the **different approaches used** (user-based, item-based, random-walk).

#### **2. Dataset Description**
- Describe the **MovieLens 100K dataset**:
  - Number of users, movies, and ratings.
  - What features were used.
  - Any preprocessing performed.

#### **3. Methodology**
- Explain the three recommendation techniques implemented:
  - **User-based collaborative filtering** (how user similarity was calculated).
  - **Item-based collaborative filtering** (how item similarity was determined).
  - **Random-walk-based Pixie algorithm** (why graph-based approaches are effective).
  
#### **4. Implementation Details**
- Discuss the steps taken to build the functions.
- Describe how the **adjacency list graph** was created.
- Explain how **random walks** were performed and how visited movies were ranked.

#### **5. Results and Evaluation**
- Present **example outputs** from each recommendation approach.
- Compare the different methods in terms of accuracy and usefulness.
- Discuss any **limitations** in the implementation.

#### **6. Conclusion**
- Summarize the key takeaways from the project.
- Discuss potential improvements (e.g., **hybrid models, additional features**).
- Suggest real-world applications of the methods used.

### **Submission Instructions**

- Submit `.zip` file consisting of Jupyter Notebook and all the datafiles (provided) and the ones saved [i.e. `users.csv`, `movies.csv` and `ratings.csv`]. Also, include the Report and Pixie Algorithm explanation document.
- [`Bonus 10 Points`] **Upload your Jupyter Notebook, Explanation Document, and Report** to your GitHub repository.
- Ensure the repository is public and contains:
  - `users.csv`, `movies.csv` and `ratings.csv` [These are the Dataframes which were created in part 1. Save and export them as a `.csv` file]
  - `Movie_Recommendation.ipynb`
  - `Pixie_Algorithm_Explanation.pdf` or `.md`
  - `Recommendation_Report.pdf` or `.md`
- **Submit the GitHub repository link in the cell below.**


#### **Example Submission Format**
```text
GitHub Repository: https://github.com/username/Movie-Recommendation
```

In [ ]:
# Submit the Github Link here:
# https://github.com/zhutchens/Movie-Recommendation

### **Grading Rubric: ITCS 6162 - Data Mining Assignment**


| **Category**                              | **Criteria**                                                     | **Points** |
|-------------------------------------------|----------------------------------------------------------------|------------|
| **Part 1: Exploring and Cleaning Data (15 pts)**  | Properly loads `u.user`, `u.movies`, and `u.item` datasets into DataFrames | 5 |
|                                           | Handles missing values, duplicates, and inconsistencies appropriately | 5 |
|                                           | Saves the cleaned datasets into CSV files: `users.csv`, `movies.csv`, `ratings.csv` | 5 |
| **Part 2: Collaborative Filtering-Based Recommendation (30 pts)** | Implements user-based collaborative filtering correctly | 10 |
|                                           | Implements item-based collaborative filtering correctly | 10 |
|                                           | Computes similarity measures accurately and provides valid recommendations | 10 |
| **Part 3: Graph-Based Recommender (Pixie-Inspired Algorithm) (35 pts)** | Constructs adjacency lists properly from user-movie interactions | 10 |
|                                           | Implements weighted random walk-based recommendation correctly | 15 |
|                                           | Explains and justifies the algorithm design choices (Pixie-inspired) | 10 |
| **Code Quality & Documentation (10 pts)** | Code is well-structured, efficient, and follows best practices | 5 |
|                                           | Markdown explanations and comments are clear and enhance understanding | 5 |
| **Results & Interpretation (5 pts)**      | Provides meaningful insights from the recommendation system's output | 5 |
| **Submission & Report (5 pts)**          | Submits all required files in the correct format (ZIP file with Jupyter notebook, processed CSV files, and project report) | 5 |
| **Total**                                 |                              | 100 |

#### **Bonus (10 pts)**
| **Category**                              | **Criteria**                                                     | **Points** |
|-------------------------------------------|----------------------------------------------------------------|------------|
| **GitHub Submission**                     | Provides a well-documented GitHub repository with CSV files, a structured README, and a properly formatted Jupyter Notebook | 10 |